In [ ]:
import scanpy as sc
import numpy as np

In [ ]:
dir_path = "/home/krushna/Documents/Data_integration/SCRNA_Datasets/All_h5ad/"
def load_data(dataset,batch):
    adata =sc.read_h5ad(dir_path+dataset+'.h5ad')
    sc.pp.filter_genes(adata, min_counts=3)
    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata
    sc.pp.highly_variable_genes(
            adata,
            flavor="seurat",
            n_top_genes=2000,
            layer="counts",
            batch_key=batch,
            subset=True
    )
    return adata
    
batch_key_dic = {'Immune_Human' : 'batch',
                 'Immune_human_mouse' : 'batch',
                 'Lung' : 'batch',
                 'Mouse_brain' : 'batch',
                 'Pancreas' : 'Batch',
                 'Simulation1' : 'Batch',
                 'Simulation2' : 'Batch'}
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Immune_human_mouse' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Mouse_brain' : 'cell_type',
                 'Pancreas' : 'Group',
                 'Simulation1' : 'Group',
                 'Simulation2' : 'Group'}

In [ ]:
dataset = 'Immune_Human'
batch = batch_key_dic[dataset]
cell_type = cell_type_key_dic[dataset]
adata = load_data(dataset,batch)
adata_sce = adata.copy()
if type(adata_sce.X) != type(np.array([])):
    adata_sce.X = adata_sce.X.toarray()
    adata_sce.layers['counts'] = adata_sce.layers['counts'].toarray()
    # del adata_sce.layers['counts']
del adata_sce.uns
del adata_sce.var

In [ ]:
print(adata,adata_sce,sep='\n')

In [ ]:
import time
start_time = time.time()

In [ ]:
import anndata2ri
anndata2ri.activate()

#Loading the rpy2 extension enables cell magic to be used
#This runs R code in jupyter notebook cells
%load_ext rpy2.ipython

sc.settings.verbosity = 3
sc.logging.print_versions()

In [ ]:
%%R -i adata_sce

library(Seurat)
adata_seu <- as.Seurat(adata_sce, counts = "counts", data = "X")


In [ ]:
%%R  -i batch
batch_list = SplitObject(adata_seu, split.by = batch)

In [ ]:
%%R
hvg=2000
anchors = FindIntegrationAnchors(
object.list = batch_list,
anchor.features = hvg,
scale = T,
l2.norm = T,
dims = 1:30,
k.anchor = 5,
k.filter = 200,
k.score = 30,
max.features = 200,
eps = 0)

In [ ]:
%%R
integrated = IntegrateData(anchorset = anchors)

In [ ]:
%%R -o integrated_sce
integrated_sce <-  as.SingleCellExperiment(integrated)

In [ ]:
adata_int = integrated_sce
adata_int

In [ ]:
end_time = time.time()
print("total time taken: ", end_time-start_time)

In [ ]:
try:
    del adata_int.obs['nFeature_originalexp']
    del adata_int.obs['nGene']
    del adata_int.obs['nUMI']
except :
    pass
adata_int.write('seurat-Immune_human.h5ad')

In [ ]:
adata_int = sc.read_h5ad('seurat-Immune_Human.h5ad')

In [ ]:
import scIB
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(
        adata,
        adata_int,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=True,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=True,
        embed= 'X_pca',
        si_metric='euclidean',
        pcr_=True,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=True,  # backwards compatibility
        isolated_labels_f1_=False,
        isolated_labels_asw_=False,
        n_isolated=None,
        graph_conn_=True,
        kBET_=True,
        kBET_sub=0.5,
        lisi_graph_=True,
        lisi_raw=True,
        trajectory_=False,
        type_=None,
        verbose=False,
)

In [ ]:
results

In [ ]:
import numpy as np
np.savetxt(dataset+"_ilisi.csv", ilisi_all, delimiter=",")
np.savetxt(dataset+"_clisi.csv", clisi_all, delimiter=",")
np.savetxt(dataset+"_kbet_all.csv",np.concatenate([np.array(val).reshape(1,-1) for val in kbet_all],axis = 0), delimiter=',')

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi


latent_matrix = adata_int.X 
labels = np.array(adata.obs[cell_type] )
K = np.size(np.unique(labels))
kmeans = KMeans(n_clusters=K, random_state=0).fit(latent_matrix)
y_pred = kmeans.labels_

print('Computing NMI ...')
NMI = nmi(labels.flatten(), y_pred.flatten())
print('NMI = {}'.format(NMI))

In [ ]:
sc.pp.neighbors(adata_int)  # use_rep = 'final_embeddings'
sc.tl.umap(adata_int)
sc.pl.umap(adata_int, color=cell_type, frameon=False)
sc.pl.umap(adata_int, color=batch, frameon=False)
sc.pl.umap(adata_int, color='cluster', frameon=False)

In [ ]:
adata_int.obs['temp'] =  y_pred
adata_int.obs['temp'] = adata_int.obs['temp'].astype(str)
sc.pl.umap(adata_int, color='temp', frameon=False)

In [ ]:
import scIB
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(
        adata,
        adata_int,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=False,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=False,
        embed= 'X_pca',
        si_metric='euclidean',
        pcr_=False,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=False,  # backwards compatibility
        isolated_labels_f1_=False,
        isolated_labels_asw_=False,
        n_isolated=None,
        graph_conn_=False,
        kBET_=False,
        kBET_sub=0.5,
        lisi_graph_=False,
        lisi_raw=False,
        trajectory_=True,
        type_=None,
        verbose=False,
)
results.loc['trajectory']

In [ ]:
import scIB
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(
        adata,
        adata_int,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=True,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=False,
        embed= 'X_pca',
        si_metric='euclidean',
        pcr_=False,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=False,  # backwards compatibility
        isolated_labels_f1_=False,
        isolated_labels_asw_=False,
        n_isolated=None,
        graph_conn_=False,
        kBET_=False,
        kBET_sub=0.5,
        lisi_graph_=False,
        lisi_raw=False,
        trajectory_=False,
        type_=None,
        verbose=False,
)
results

In [ ]:
sc.settings.figdir = '/home/krushna/Documents/Data_integration/Figures/Immune_Human/'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
sc.pp.neighbors(adata_int)  # use_rep = 'final_embeddings'
sc.tl.umap(adata_int)

In [ ]:
sc.pl.umap(adata_int, color=cell_type, frameon=False,save='_seurat_'+dataset+"_cell_type.png")
sc.pl.umap(adata_int, color=batch, frameon=False,save='_seurat_'+dataset+"_batch.png")
sc.pl.umap(adata_int, color='cluster', frameon=False,save='_seurat_'+dataset+"_optimal_resolution.png")

In [ ]:
import pandas as pd
import numpy as np
cluster_cell_type = pd.crosstab(np.array(adata_int.obs['cluster']),np.array(adata_int.obs['final_annotation']))
cluster_cell_type.index.name = 'cluster'
cluster_cell_type

In [ ]:
li = []
for column in cluster_cell_type:
    li.append([column,entropy(cluster_cell_type[column])])
pd.DataFrame(li)